In [1]:
import requests
from lxml import html
import re
import redis
from urllib import parse
import time
import random
import json
import pymongo

In [2]:
redis_client = redis.Redis(host="li.goldlone.cn", port=6379, db=0)

In [3]:
# mongo_client = pymongo.MongoClient(host='li.goldlone.cn',port=27017)

In [4]:
# mongo_db = mongo_client["jobs"]
# mongo_col_qiancheng = mongo_db["qiancheng"]

In [4]:
# url = "https://jobs.51job.com/beijing-cyq/101450454.html?s=01&t=0"
# r = requests.get(url, headers=headers)

In [2]:
key = '大数据'

redis_url_ok_key = "jobs:qiancheng51_url_ok"
redis_url_need_key = "jobs:qiancheng51_url_need"

headers = {
    'Host': 'search.51job.com',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
}

In [6]:
def get_links(keywords, page):
    '''
    获取查询页的所有链接
    '''
    key = parse.quote(parse.quote(keywords))
    url = 'http://search.51job.com/list/000000,000000,0000,00,9,99,' + key + ',2,' + str(page) + '.html'

    try:
        r = requests.get(url, headers=headers, timeout=10)
        s = requests.session()
        s.keep_alive = False
        r.encoding = 'gbk'
        reg = re.compile(r'class="t1 ">.*? <a target="_blank" title=".*?" href="(.*?)".*? <span class="t2">', re.S)
        links = re.findall(reg, r.text)
    except Exception as e:
        print(e)
        return get_links(page)

    return links

In [7]:
links = get_links("大数据", 1)

In [10]:
def get_content(link):
    '''
    获取详情页的具体信息
    '''
    sub_link = link[0 : link.index("?")]
    if not redis_client.sismember(redis_url_ok_key, sub_link):
        r = requests.get(sub_link, headers=headers, timeout=10)
        s = requests.session()
        s.keep_alive = False
        r.encoding = 'gbk'

        t = html.fromstring(r.text)
        try:
            # 过滤无效字符
            pattern = re.compile(r'[\r|\t|\n|\xa0]')

            # 岗位名称
            title = t.xpath("/html/body/div[3]/div[2]/div[2]/div/div[1]/h1/@title")[0]
            # 月薪
            salary = t.xpath("/html/body/div[3]/div[2]/div[2]/div/div[1]/strong/text()")[0]

            # 关键词
            keywords = t.xpath("/html/body/div[3]/div[2]/div[2]/div/div[1]/p[2]/text()")
#             address = re.sub(pattern, "", keywords[0])
#             experience = re.sub(pattern, "", keywords[1])
#             education = re.sub(pattern, "", keywords[2])
#             num = re.sub(pattern, "", keywords[3])
#             date = re.sub(pattern, "", keywords[4])
            keywords = re.sub(pattern, "", ",".join(keywords))

            # 岗位特色
            features = t.xpath("/html/body/div[3]/div[2]/div[2]/div/div[1]/div/div/span/text()")
            features = ",".join(features)

            # 岗位描述
            description = t.xpath("/html/body/div[3]/div[2]/div[3]/div[1]/div/p/text()")
            description = " ".join(description)

            # 公司名称
            company_name = t.xpath("/html/body/div[3]/div[2]/div[4]/div[1]/div[1]/a/p/@title")[0]
            # 公司信息
            company_infos = t.xpath("/html/body/div[3]/div[2]/div[4]/div[1]/div[2]/p/@title")
            # 公司类型
            company_type = company_infos[0]
            # 公司规模
            company_scala = company_infos[1]
            # 公司所处行业
            company_trade = company_infos[2]
            # 公司描述
            company_description = t.xpath("/html/body/div[3]/div[2]/div[3]/div[3]/div/text()")
            company_description = re.sub(pattern, "", " ".join(company_description))

            info = {
#                 "address": address,
#                 "experience": experience,
#                 "education": education,
#                 "num": num,
#                 "date": date,
                "title": title,
                "salary": salary,
                "keywords": keywords,
                "features": features,
                "description": description,
                "company_name": company_name,
                "company_type": company_type,
                "company_scala": company_scala,
                "company_trade": company_trade,
                "company_description": company_description,
                "url": sub_link
            }

            # 存储到文件
            with open("./qiancheng51.txt", 'a+', encoding='utf-8') as file:
                file.write(json.dumps(info, ensure_ascii=False) + '\n')
            # /html/body/div[3]/div[2]/div[4]/div[2]/div[2]/div[1]/a[1]
            
            # 存储到MongoDB
#             mongo_col_qiancheng.insert_one(info)
            
            # 保存相似岗位的链接
            similar_job_urls = t.xpath("/html/body/div[3]/div[2]/div[4]/div[2]/div[2]/div/a[1]/@href")
            for link in similar_job_urls:
                redis_client.lpush(redis_url_need_key, link)

        except Exception as e:
            print("解析或存储异常<{}>: ".format(sub_link), e)
            return False
        
        redis_client.sadd(redis_url_ok_key, sub_link)
        print("success crawl url: ", sub_link)
        return True
    else:
        print("重复URL<{}>".format(sub_link))
        return False

In [9]:
url = "https://jobs.51job.com/beijing-cyq/101450454.html?s=01&t=0"
r = requests.get(url, headers=headers)

In [16]:
t = html.fromstring(r.text)
similar_job_urls = t.xpath("/html/body/div[3]/div[2]/div[4]/div[2]/div[2]/div/a[1]/@href")
# pattern = re.compile(r'[\r|\t|\n|\xa0]')
# re.sub(pattern, "", ",".join(keywords))
similar_job_urls

['https://jobs.51job.com/beijing-hdq/107589625.html?t=2&s=01',
 'https://jobs.51job.com/beijing-hdq/108008724.html?t=2&s=01',
 'https://jobs.51job.com/beijing-hdq/95876778.html?t=2&s=01',
 'https://jobs.51job.com/beijing-cyq/100133804.html?t=2&s=01',
 'https://jobs.51job.com/beijing/105340991.html?t=2&s=01',
 'https://jobs.51job.com/beijing/89414954.html?t=2&s=01',
 'https://jobs.51job.com/beijing-hdq/108363076.html?t=2&s=01',
 'https://jobs.51job.com/beijing-cyq/106184397.html?t=2&s=01',
 'https://jobs.51job.com/beijing-cyq/106767791.html?t=2&s=01',
 'https://jobs.51job.com/beijing-dcq/95363288.html?t=2&s=01']

In [157]:
for link in links:
    redis_client.lpush(redis_url_need_key, link)
#     get_content(link)
#     time.sleep(random.randint(5, 10))
#     break

In [14]:
# 从待爬取队列中拉
# while True:
link_data = redis_client.brpop(redis_url_need_key)
link = link_data[1].decode()

get_content(link)

success crawl url:  https://jobs.51job.com/guangzhou/101930305.html


True

In [152]:
links

['https://jobs.51job.com/shanghai/108326727.html?s=01&t=0',
 'https://jobs.51job.com/shanghai-pdxq/80446141.html?s=01&t=0',
 'https://jobs.51job.com/wuhan-dhxjs/109546711.html?s=01&t=0',
 'https://jobs.51job.com/shenzhen-nsq/109544219.html?s=01&t=0',
 'https://jobs.51job.com/chengdu-gxq/109076054.html?s=01&t=0',
 'https://jobs.51job.com/shanghai-ypq/108832468.html?s=01&t=0',
 'https://jobs.51job.com/shanghai-pdxq/109539342.html?s=01&t=0',
 'https://jobs.51job.com/guangzhou-pyq/109538256.html?s=01&t=0',
 'https://jobs.51job.com/qingdao-hdq/109536026.html?s=01&t=0',
 'https://jobs.51job.com/hangzhou-xhq/109535680.html?s=01&t=0',
 'https://jobs.51job.com/beijing-hdq/109534652.html?s=01&t=0',
 'https://jobs.51job.com/chengdu/109534106.html?s=01&t=0',
 'https://jobs.51job.com/changsha/109533698.html?s=01&t=0',
 'https://jobs.51job.com/beijing-hdq/109532073.html?s=01&t=0',
 'https://jobs.51job.com/shanghai-jaq/109531101.html?s=01&t=0',
 'https://jobs.51job.com/zhenjiang/109530596.html?s=01&t

In [3]:
url = "https://jobs.51job.com/shenzhen-ftq/108815735.html"
r = requests.get(url, headers=headers, timeout=10)
t = html.fromstring(r.text)

In [11]:
t.xpath("/html/body/div[3]/div[2]/div[2]/div/div[1]/strong/text()")

<Element html at 0x1cd71f1ea98>